In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score as r2


import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm

import requests
import json
from datetime import datetime
import os

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [19]:
BASE_URL = 'http://192.168.1.222:8999'

print(requests.get(f'{BASE_URL}/').text)

current_datetime = datetime.now().strftime('%Y-%m-%d-%H-%M')

# JSONファイルの読み込み
with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

# 全てのシンボルをプリント
for category in data:
    for company in data[category]:


        stock_code = company["symbol"]
        print(stock_code)
        
        df = requests.get(f'{BASE_URL}/ml_data/{stock_code}').json()
        df = pd.DataFrame(df)
        print(df.info())
        

Hello Server

A
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NY_Dow               2999 non-null   float64
 1   SP_500               2999 non-null   float64
 2   content_concern      2999 non-null   float64
 3   content_despair      2999 non-null   float64
 4   content_excitement   2999 non-null   float64
 5   content_optimism     2999 non-null   float64
 6   content_stability    2999 non-null   float64
 7   date                 2999 non-null   object 
 8   headline_concern     2999 non-null   float64
 9   headline_despair     2999 non-null   float64
 10  headline_excitement  2999 non-null   float64
 11  headline_optimism    2999 non-null   float64
 12  headline_stability   2999 non-null   float64
 13  time                 2999 non-null   object 
 14  value                2999 non-null   float64
 15  vix                  2

KeyboardInterrupt: 